## Role Models
Presented by Aldrin G. Hsu, Wesley Brian T. Sy, and Gian Martin C. Carlos



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
dfIncome = pd.read_excel("Table 6. 2018, 2021 and 2023p Share to Total Annual Family Income, by Region, Province and HUC.xlsx", engine="openpyxl")
dfExpenditure = pd.read_excel("Table 7. 2018, 2021 and 2023p Share to Total Family Expenditure, by Region, Province and HUC.xlsx", engine="openpyxl")

In [6]:
dfExpenditure.head(10)


,"Table 7. Share to Total Family Expenditure, by Region, Province and HUC: 2018, 2021 and 2023p",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Region/Province/HUC by Expenditure Group*,2018,2021.00,2023p,2021-2018,2023-2021
1,NaN,(In percent),NaN,NaN,Percentage Point Change,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,Philippines,NaN,NaN,NaN,NaN,NaN
4,Share to Family Expenditure,100,100.00,100,NaN,NaN
5,Food and Non-Alcoholic Beverages,42.55,42.62,40.93,0.07,-1.7
6,Food Consumed at Home,33.6,38.53,35.34,4.93,-3.19
7,Food Regularly Consumed Outside the Home,8.95,4.09,5.57,-4.86,1.49
8,Service for Processing Primary Goods for F...,NaN,0.01,0.01,0.01,0
9,Alcoholic Beverages and Tobacco,1.82,1.74,1.42,-0.09,-0.32


In [ ]:
#Table 6. 2018, 2021 and 2023p Share to Total Annual Family Income, by Region, Province and HUC
Income18 = pd.read_csv("I18.csv")
Income21 = pd.read_csv("I21.csv")
Income23 = pd.read_csv("I23.csv")

#Table 7. 2018, 2021 and 2023p Share to Total Family Expenditure, by Region, Province and HUC
Expenditure18 = pd.read_csv("E18.csv")
Expenditure21 = pd.read_csv("E21.csv")
Expenditure23 = pd.read_csv("E23.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'I18'